# Small consumer 2-zone test

Using generated price timeseres, assume that national grid system is beyond consumer's control.

What we then need to do is to introduce a second zone (or “bus”) representing consumer’s assets

1. Establish a second “zone” with a single connector between it and the first zone.
2. The second zone should contain consumer’s:
   a. demand for power ... using `outputs_plan_national-grid/???.csv`, rescale National Grid mean & std. to 1%, and use to generate a random timeseries of sorm form
   b. diesel generators ... (for now) set the *capacity* of these generators to be effectively infinite, but the consumption cost to be *very high* (say 2-3 times that of the peaking plant in the national grid zone but not as high as unmet demand).  
3. The connector between the “zones” should be:
   a. Effectively infinite in capacity when taking power from the national grid zone to the consumer zone
   b. Zero cost per unit of power transferred
   c. Unidirectional (only takes power from Nat Grid to consumer, not vice versa). see https://calliope.readthedocs.io/en/stable/user/advanced_constraints.html#one-way-transmission-links

Big picture: larger zone (National Grid) sets its own prices based on national demand/wind etc. The smaller zone is always able to meet it’s own internal demand if it chooses to (diesel capacity is very large) but would usually *prefer* to take power from national grid rather than use it’s own diesel generators. No power export from consumer to grid.

Further work:

* Shrink national grid generation capacities (introduce shortages), observe impact on consumer zone.
* Look at swapping diesel generators to a *storage* generator(s).

In [ ]:
# Suppress minor warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import calliope
import models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Read in National grid planning output (full 1980-2017)

From timeseries calculate mean and standard deviation of demand.
Change back from negative convention (calliope) to positive demand convention (readable)

In [ ]:
# read csv
df = pd.read_csv('outputs_plan_national-grid/inputs_resource.csv',
                 usecols=['timesteps', 'resource', 'techs'], index_col='timesteps')
# split demand / wind /solar into 3 separate columns
national_demand_index = pd.to_datetime(df[df['techs'] == 'demand_power'].index)
national_demand = pd.DataFrame(dict(), index=national_demand_index)
national_demand['demand'] = - df[df['techs'] == 'demand_power']['resource']
national_demand['wind'] = df[df['techs'] == 'wind']['resource']
national_demand['solar'] = df[df['techs'] == 'solar']['resource']
del df

In [ ]:
national_demand_stats = mean, std = [stat(national_demand.demand) for stat in [np.mean, np.std]]

### Generate a random timeseries 
form with similar overall mean/stddev to UK wide demand (scaled to 1%), ensure no negative values

In [ ]:
# ensure always using the same seed (which we obtained one-off using RNG)
np.random.seed(285183912)

# normal distribution parameters = 1% National Grid mean/std dev.
normal_dist = [stat*0.01 for stat in national_demand_stats]

# Sample from normal distribution
demand_region2 = np.random.normal(*normal_dist, len(national_demand))

# Force positives (very minute chance of this occuring...)
demand_region2[demand_region2 < 0] = 0

In [ ]:
demand_region2

### Load secondary zone demand into a DataFrame
Allows easy loading into ts_data

In [ ]:
df_region2 = pd.DataFrame({'demand_region2': demand_region2},
                          index=pd.to_datetime(national_demand.index))

# `operational` Calliope 2-zone model (with infinite diesel cap)

Note: can shrink operational range to subset, eg. 2017 year only

In [ ]:
date_start, date_end = '1980', '2017'

In [ ]:
# Import timeseries data demand / wind (as in 1_region)
ts_data = models.load_time_series_data('2_region', additional_data=df_region2)

# Crop to date range
ts_data = ts_data.loc[date_start:date_end]

display(ts_data.head(6))

In [ ]:
# Read in generation capacities
generation_capacities_planned = pd.read_csv('outputs_plan_national-grid/results_energy_cap.csv')

# Rename techs
generation_capacities = dict()
for tech, cap in zip(generation_capacities_planned['techs'],
                     generation_capacities_planned['energy_cap']):
    if tech not in ['unmet', 'demand_power']:
        key = f'cap_{tech}_region1'
        generation_capacities[key] = cap

# Insert diesel generators, transmission capacity = max possible region2 demand
generation_capacities['cap_generators_region2'] = max(ts_data['demand_region2'])
generation_capacities['cap_transmission_region1_region2'] = max(ts_data['demand_region2'])


# Display
display(generation_capacities)

In [ ]:
# Create the model with fixed capacities
model = models.TwoRegionModel(ts_data, 'operate', fixed_caps=generation_capacities)
display(model.preview('inputs.resource'))

In [ ]:
model.run()
model.get_summary_outputs()

In [ ]:
def cap_mean_plot(model, subtitle: str=None, fname: str=None):
    # Extract mean capacity factors from model
    CF = dict(zip(model.results.capacity_factor.loc_tech_carriers_prod.values,
                  np.mean(model.results.capacity_factor.values, axis=1)))

    # setup figure axs and title
    fig, axs = plt.subplots(3, 3, figsize=(12,9))
    title = 'Capacity Factor Means'
    if subtitle:
        title += '\n' + subtitle
    fig.suptitle(title)
    
    # pie chart for each tech
    axs_idx = 0
    for tech, cap in CF.items():
        axs_pos = (axs_idx%3, axs_idx//3)
        label = tech[:-7].replace('::','\n').replace('transmission_', 'transmission\n').replace(':region1','')
        label += f'\n{cap:.3f}'
        axs[axs_pos].pie([cap, 1-cap], labels=[label,None])
        axs_idx += 1

    # plot legend
    fig.subplots_adjust(top=0.95, bottom=0.05, right=0.95, left=0.05, hspace=0.01, wspace=0.15)
    axs[-1, -1].pie([0,0], labels=[r'$\mu$', r'1-$\mu$'])
    plt.legend()

    # save plot
    if fname:
        plt.savefig('plots/'+fname, dpi=300)

cap_mean_plot(model)

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_operate')
output_folder = 'outputs_operational_two-zone'
models.rm_folder(output_folder)
model.to_csv(output_folder)

In [ ]:
# Generate HTML plots
for var in ['power', 'cost_var', 'resource']:
    plot_html = model.plot.timeseries(array=var, html_only=True)
    models.save_html(plot_html, f'plots/operate_2_{var}.html', f'{var} plot')

In [ ]:
model.preview('results.systemwide_capacity_factor', loc='carriers', time_idx=False,
              index=model.results.systemwide_capacity_factor.techs.values)

### Adjust Generators Operating cost

Trial different operating costs, and get the capacity factor

In [ ]:
# Create the model with fixed capacities
# change the generator operating cost
mod = dict()

# sub baseload cost
print('generators om_con < baseload om_con')
mod['sb'] = models.TwoRegionModel(ts_data, 'operate', fixed_caps=generation_capacities, extra_override='generator_cost_sb')
# sub peaking cost
print('generators om_con < peaking om_con')
mod['sp'] = models.TwoRegionModel(ts_data, 'operate', fixed_caps=generation_capacities, extra_override='generator_cost_sp')
# sub unmet cost
print('generators om_con < unmet om_con')
mod['su'] = models.TwoRegionModel(ts_data, 'operate', fixed_caps=generation_capacities, extra_override='generator_cost_su')

for name, model in mod.items():
    print(f'running {name} model')
    model.run()

In [ ]:
# Print out capacity factors
model_example = list(mod.values())[0]
columns = [s.split('::')[1] for s in model_example.results.capacity_factor.loc_tech_carriers_prod.values]
cap_factors = pd.DataFrame(columns=columns)

for name, model in mod.items():
    cap_factors = cap_factors.append(dict(zip(columns, np.mean(model.results.capacity_factor.values, axis=0))), ignore_index=True)

print('Capacity factors under each scenario:')
cap_factors.index = list(mod.keys())
display(cap_factors)

In [ ]:
# calculate prices ???

In [ ]:
# memory clearing
del mod

### Induce National Grid Shortages

With the original `om_con=0.1` for generators, trial a range of reduced capacity national grid operation

In [ ]:
# Create the model with fixed capacities
# change the baseload and peaking capacities
mod = dict()

# shortcut for baseload and peaking capacities
baseload = generation_capacities['cap_baseload_region1']
peaking = generation_capacities['cap_peaking_region1']

# reduced ALL region1 capacities by X%
for reduction in range(0, 31, 5):
   reduction_percent = 1 - reduction / 100
   caps = {tech: val if 'region2' in tech else val*reduction_percent
           for tech, val in generation_capacities.items()}
   mod[reduction] = models.TwoRegionModel(ts_data, 'operate', fixed_caps=caps)

for name, model in mod.items():
    print(f'running {name} model')
    model.run()

In [ ]:
# Print out capacity factors
model_example = list(mod.values())[0]
columns = [s.split('::')[1] for s in model_example.results.capacity_factor.loc_tech_carriers_prod.values]
cap_factors = pd.DataFrame(columns=columns)

for name, model in mod.items():
    cap_factors = cap_factors.append(dict(zip(columns, np.mean(model.results.capacity_factor.values, axis=0))), ignore_index=True)

print('Capacity factors under each scenario:')
cap_factors.index = list(mod.keys())
display(cap_factors)

In [ ]:
# calculate prices ???
import IPython; IPython.embed()

In [ ]:
# memory clearing
del mod